# 3. Generate Audio (HD Voice)

https://learn.microsoft.com/en-us/azure/ai-services/speech-service/high-definition-voices

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

print(os.getenv("AZURE_OPENAI_ENDPOINT")) 
print(os.getenv("AZURE_OPENAI_MODEL"))

In [ ]:
import json

with open("status-report-1-script.json","r") as f:
    jsfile = f.read()

conversation = json.loads(jsfile)
conversation

## HD Options

Default [temparature parameter](https://learn.microsoft.com/en-us/azure/ai-services/speech-service/high-definition-voices#how-to-use-azure-ai-speech-hd-voices) is 1.0

```
<voice name='en-us-Andrew2:DragonHDLatestNeural' parameters='temperature=0.1'>저도 감사합니다, 제인님. 다음에 또 뵙길 바랍니다.</voice>
```

In [ ]:
def generate_ssml(host_voice, guest_voice):
    
    ssml = "<speak version='1.0' xmlns='http://www.w3.org/2001/10/synthesis' xmlns:mstts='https://www.w3.org/2001/mstts' xml:lang='en-US'>"
    for r in conversation:
        #print(row.to_dict())
        
        if r['speaker'] == 'Host (Jane)':
            ssml += f"\n<voice name='{host_voice}'>{r['text']}</voice>"
        else:
            ssml += f"\n<voice name='{guest_voice}'>{r['text']}</voice>"
    ssml += "\n</speak>"

    #print(ssml)

    return ssml

In [ ]:
import azure.cognitiveservices.speech as speechsdk
import os
speech_key = os.getenv('SPEECH_KEY')
service_region = os.getenv('SPEECH_REGION')
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

In [ ]:
import random
import string

def generate_random_filename(length=8):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_podcast(ssml, filename=None):
    if filename is None:
        temporary_file= "./" + generate_random_filename() + ".wav"
    else:
        temporary_file = filename
        
    audio_output = speechsdk.audio.AudioOutputConfig(filename=temporary_file)

    # Creates a speech synthesizer using the Azure Speech Service.
    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_output)
    result = speech_synthesizer.speak_ssml_async(ssml).get()
    if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print("Speech synthesis was successful. Audio was written to '{}'".format(temporary_file))
    elif result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = result.cancellation_details
        print("Speech synthesis canceled: {}".format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print("Error details: {}".format(cancellation_details.error_details))
        print("Did you update the subscription info?")

In [ ]:
# HD voice
host_voice = 'en-us-Ava:DragonHDLatestNeural'
guest_voice = 'en-us-Andrew2:DragonHDLatestNeural'


In [ ]:
temp = 0.9
ssml = generate_ssml(host_voice, guest_voice)
generate_podcast(ssml, "Progress_report_podcast_hd.wav")

In [ ]:
# Non-HD voice
#host_voice = "ko-KR-SunHiNeural"
#guest_voice = "ko-KR-InJoonNeural"